In [1]:
# Load data from Excel file
def GetData(DataFile, DataWorksheet):
    Width = LoadFromExcel(DataFile, DataWorksheet, 'Width')
    Length = LoadFromExcel(DataFile, DataWorksheet, 'Length')
    Weight = LoadFromExcel(DataFile, DataWorksheet, 'Weight')
    Width.columns = ['Item']
    Length.columns = ['Item']
    Weight.columns = ['Item']
    return Width, Length, Weight

In [2]:
# Define model data, assigning all data to the Model
def DefineModelData(Model, Width, Length, Weight):
    Model.Item = pyo.Set(initialize = range(0, len(Width)))
    Model.Candidate = pyo.Set(initialize = range(0, 3 * len(Width) * len(Width)))
    Model.Width = pyo.Param(Model.Item, within = pyo.NonNegativeIntegers, mutable = True)
    Model.Length = pyo.Param(Model.Item, within = pyo.NonNegativeIntegers, mutable = True)
    Model.Weight = pyo.Param(Model.Item, within = pyo.NonNegativeReals, mutable = True)
    Model.BigM = pyo.Param(Model.Item, within = pyo.NonPositiveReals, mutable = True)   # Note domain reflects expectation of negative value

    Model.Baseline = 0   # Total weighted area of all items
    for i in Model.Item:
        Model.Width[i] = Width['Item'][i]
        Model.Length[i] = Length['Item'][i]
        Model.Weight[i] = Weight['Item'][i]
        Model.Baseline += Model.Width[i] * Model.Length[i] * Model.Weight[i]
        
    # Define candidate product sizes using width and length of items, taking item width and lengths independently to enumerate all combinations
    Model.CandidateWidth = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)
    Model.CandidateLength = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)
    Model.CandidateArea = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)
    for i in Model.Item:   # All widths x lengths
        for j in Model.Item:
            Model.CandidateWidth[i * len(Width) + j] = Width['Item'][i]
            Model.CandidateLength[i * len(Width) + j] = Length['Item'][j]
            Model.CandidateArea[i * len(Width) + j] = Width['Item'][i] * Length['Item'][j]
    for i in Model.Item:   # All widths x widths
        for j in Model.Item:
            Model.CandidateWidth[(len(Width) * len(Width)) + i * len(Width) + j] = Width['Item'][i]
            Model.CandidateLength[(len(Width) * len(Width)) + i * len(Width) + j] = Width['Item'][j]
            Model.CandidateArea[(len(Width) * len(Width)) + i * len(Width) + j] = Width['Item'][j] * Width['Item'][i]
    for i in Model.Item:   # All lengths x lengths
        for j in Model.Item:
            Model.CandidateWidth[(2 * len(Width) * len(Width)) + i * len(Width) + j] = Length['Item'][i]
            Model.CandidateLength[(2 * len(Width) * len(Width)) + i * len(Width) + j] = Length['Item'][j]
            Model.CandidateArea[2 * (len(Width) * len(Width)) + i * len(Width) + j] = Length['Item'][j] * Length['Item'][i]
            
    for i in Model.Item:
        Model.BigM[i] = -abs(Width['Item'][i] - Length['Item'][i])  # Individual Big M for each item. Note the negative sign